# Data Handling
***

Some datasets contain more than just numerical data.
In this notebook we discuss how to handle **categorical data**.

This time, we will load a `.csv` file.
You can find it on the grips page of this course.
This time it is an adult census dataset.
The task associated to this dataset is a classification task.

In [9]:
import pandas as pd

adult_census = pd.read_csv("~/Documents/ML2023/Ex091123/adult-census.csv")

In [10]:
adult_census.head()

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


The columns `education` and `education-num` contain the same information so we delete it.

In [11]:
adult_census = adult_census.drop(columns="education-num")

adult_census.head()

,age,workclass,education,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,25,Private,11th,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,HS-grad,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,Assoc-acdm,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,Some-college,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,Some-college,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


As mentioned above, there is a classification problem associated to this dataset: decide whether or not a person earns more than 50k a year. So we need to split the target from this dataset.

In [12]:
target_name = "class"
target = adult_census[target_name]

data = adult_census.drop(columns=[target_name])

This dataset not only contains numerical variables, which are naturally handled by machine learning algorithms, but also **categorical variables**.
Categorical variables have discrete values, typically represented by string labels (but not only) taken from a finite list of possible choices.
For instance, let's look at the variable `native-country`.

In [13]:
data["native-country"].value_counts().sort_index()

native-country
 ?                               857
 Cambodia                         28
 Canada                          182
 China                           122
 Columbia                         85
 Cuba                            138
 Dominican-Republic              103
 Ecuador                          45
 El-Salvador                     155
 England                         127
 France                           38
 Germany                         206
 Greece                           49
 Guatemala                        88
 Haiti                            75
 Holand-Netherlands                1
 Honduras                         20
 Hong                             30
 Hungary                          19
 India                           151
 Iran                             59
 Ireland                          37
 Italy                           105
 Jamaica                         106
 Japan                            92
 Laos                             23
 Mexico                

Here, the value `?` indicates a missing entry.

To easily identify categorical data we can look at the data type of the columns.

In [14]:
data.dtypes

age                int64
workclass         object
education         object
marital-status    object
occupation        object
relationship      object
race              object
sex               object
capital-gain       int64
capital-loss       int64
hours-per-week     int64
native-country    object
dtype: object

Now we can select features based on their data type. This could be done manually but `scikit-learn` provides a helper function `make_column_selector`.

In [15]:
from sklearn.compose import make_column_selector as selector

categorical_columns_selector = selector(dtype_include=object)
categorical_columns = categorical_columns_selector(data)

categorical_columns

['workclass',
 'education',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'native-country']

We can now filter out columns.

In [16]:
data_categorical = data[categorical_columns]

data_categorical

,workclass,education,marital-status,occupation,relationship,race,sex,native-country
0,Private,11th,Never-married,Machine-op-inspct,Own-child,Black,Male,United-States
1,Private,HS-grad,Married-civ-spouse,Farming-fishing,Husband,White,Male,United-States
2,Local-gov,Assoc-acdm,Married-civ-spouse,Protective-serv,Husband,White,Male,United-States
3,Private,Some-college,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,United-States
4,?,Some-college,Never-married,?,Own-child,White,Female,United-States
...,...,...,...,...,...,...,...,...
48837,Private,Assoc-acdm,Married-civ-spouse,Tech-support,Wife,White,Female,United-States
48838,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,United-States
48839,Private,HS-grad,Widowed,Adm-clerical,Unmarried,White,Female,United-States
48840,Private,HS-grad,Never-married,Adm-clerical,Own-child,White,Male,United-States


We now present different strategies to encode categorical into numerical data.

## Encoding ordinal categories

One intuitive strategy is simply encoding each category with a different number. The function `OrdinalEncoder` does exactly this. Let's look at an example.

In [19]:
from sklearn.preprocessing import OrdinalEncoder

education_column = data_categorical[["education"]]

education_column

,education
0,11th
1,HS-grad
2,Assoc-acdm
3,Some-college
4,Some-college
...,...
48837,Assoc-acdm
48838,HS-grad
48839,HS-grad
48840,HS-grad


In [20]:
encoder = OrdinalEncoder().set_output(transform="pandas")

education_encoded = encoder.fit_transform(education_column)

education_encoded

,education
0,1.0
1,11.0
2,7.0
3,15.0
4,15.0
...,...
48837,7.0
48838,11.0
48839,11.0
48840,11.0


We see that each category in `"education"` has been replaced by a numerical value.
We can check the mapping between categories and numerical values by checking the fitted attribute `categories_`

In [22]:
encoder.categories_

[array([' 10th', ' 11th', ' 12th', ' 1st-4th', ' 5th-6th', ' 7th-8th',
        ' 9th', ' Assoc-acdm', ' Assoc-voc', ' Bachelors', ' Doctorate',
        ' HS-grad', ' Masters', ' Preschool', ' Prof-school',
        ' Some-college'], dtype=object)]

Now let's transform our dataset.

In [25]:
data_encoded = encoder.fit_transform(data_categorical)
data_encoded.head()

,workclass,education,marital-status,occupation,relationship,race,sex,native-country
0,4.0,1.0,4.0,7.0,3.0,2.0,1.0,39.0
1,4.0,11.0,2.0,5.0,0.0,4.0,1.0,39.0
2,2.0,7.0,2.0,11.0,0.0,4.0,1.0,39.0
3,4.0,15.0,2.0,7.0,0.0,2.0,1.0,39.0
4,0.0,15.0,4.0,0.0,3.0,4.0,0.0,39.0


We can simply also apply our encode on the whole dataset.

In [26]:
data_encoded = encoder.fit_transform(data)
data_encoded.head()

,age,workclass,education,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,8.0,4.0,1.0,4.0,7.0,3.0,2.0,1.0,0.0,0.0,39.0,39.0
1,21.0,4.0,11.0,2.0,5.0,0.0,4.0,1.0,0.0,0.0,49.0,39.0
2,11.0,2.0,7.0,2.0,11.0,0.0,4.0,1.0,0.0,0.0,39.0,39.0
3,27.0,4.0,15.0,2.0,7.0,0.0,2.0,1.0,98.0,0.0,39.0,39.0
4,1.0,0.0,15.0,4.0,0.0,3.0,4.0,0.0,0.0,0.0,29.0,39.0


Note that this encoding imposes an ordering, depending on the algorithm this may or may not matter.
For example in linear regression this poses a problem unless there is some ordering on the categorical values. On the other hand for tree based models this will not have an impact.

## Encoding nominal categories

The `OneHotEncoder` is an alternative encoding method which prevents the model to make false assumptions on the ordering of categories. 
For a given feature, it encodes as many new columns as there are possible categories. 
For a given sample, the value of the column corresponding to its category is 1, while all the columns of the other categories are set to 0. 
Let's again look at an example.

In [27]:
from sklearn.preprocessing import OneHotEncoder

hot_encoder = OneHotEncoder(sparse_output=False).set_output(transform="pandas")
education_encoded = hot_encoder.fit_transform(education_column)

education_encoded

,education_ 10th,education_ 11th,education_ 12th,education_ 1st-4th,education_ 5th-6th,education_ 7th-8th,education_ 9th,education_ Assoc-acdm,education_ Assoc-voc,education_ Bachelors,education_ Doctorate,education_ HS-grad,education_ Masters,education_ Preschool,education_ Prof-school,education_ Some-college
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
48838,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
48839,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
48840,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
